In [1]:
import time #ใช้สำหรับ เป็นตัววัดว่า อันไหนไวกว่ากัน
import numpy as np
import tensorflow as tf

In [2]:
#load data และเก็บเข้าตัวแปร
(X_train,y_train),(X_test,y_test) = tf.keras.datasets.mnist.load_data() 


In [3]:
#normallize data
X_train = X_train / 255.0
X_test = X_test / 255.0

In [4]:
#ดูขนาดมัน
X_train.shape

(60000, 28, 28)

In [5]:
#แปลงขนาดมัน
X_train = X_train.reshape(-1,28*28)
X_test= X_test.reshape(-1,28*28)

In [6]:
#ดูขนาดมันอีกที่ว่าได้แปลงรึยัง
X_train.shape

(60000, 784)

# เนื่องจากนี้จะเป็นการทดสอบประสิืทธิภาพของ distributed training
## จึงจะทำการสร้างโมเดลสองรูปแบบ แบบธรรมดา และ แบบ distributed

In [9]:
##แบบธรรมดา

normal_model = tf.keras.models.Sequential()


In [11]:
#add layer ตามปกติ
normal_model.add(tf.keras.layers.Dense(units=128, activation='relu', input_shape=(784,)))
normal_model.add(tf.keras.layers.Dropout(rate=0.2))
normal_model.add(tf.keras.layers.Dense(units=10, activation='softmax'))
normal_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])

In [12]:
#สร้างตัวแปร distubuted และกำหนด แผนการ distubuted โดยกรณีนี้เราเลือก แบบ mirror distubuted
distribute = tf.distribute.MirroredStrategy()

In [13]:
with distribute.scope():
    
    model_distributed = tf.keras.models.Sequential()
    model_distributed.add(tf.keras.layers.Dense(units=128, activation='relu', input_shape=(784,)))
    model_distributed.add(tf.keras.layers.Dropout(rate=0.2))
    model_distributed.add(tf.keras.layers.Dense(units=10, activation='softmax'))
    model_distributed.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


In [14]:
#ทดสอบ
start_time = time.time()
model_distributed.fit(X_train, y_train, epochs=10, batch_size=25)
print("Distributed training took: {} sec".format(time.time() - start_time))

Train on 60000 samples
Epoch 1/10
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
60000/60000 [==============================] - 9s 142us/sample - loss: 0.2822 - sparse_categorical_accuracy: 0.9183
Epoch 2/10
60000/60000 [==============================] - 5s 82us/sample - loss: 0.1369 - sparse_categorical_accuracy: 0.9588
Epoch 3/10
60000/60000 [==============================] - 5s 84us/sample - loss: 0.1044 - sparse_categorical_accuracy: 0.9678
Epoch 4/10
60000/60000 [==============

In [16]:
start_time = time.time()
normal_model.fit(X_train, y_train, epochs=10, batch_size=25)
print("Normal training took: {} sec".format(time.time() - start_time))

Train on 60000 samples
Epoch 1/10
60000/60000 [==============================] - 5s 78us/sample - loss: 0.2866 - sparse_categorical_accuracy: 0.9156
Epoch 2/10
60000/60000 [==============================] - 4s 73us/sample - loss: 0.1375 - sparse_categorical_accuracy: 0.9597
Epoch 3/10
60000/60000 [==============================] - 4s 73us/sample - loss: 0.1033 - sparse_categorical_accuracy: 0.9683
Epoch 4/10
60000/60000 [==============================] - 4s 72us/sample - loss: 0.0850 - sparse_categorical_accuracy: 0.9740
Epoch 5/10
60000/60000 [==============================] - 4s 73us/sample - loss: 0.0723 - sparse_categorical_accuracy: 0.9770
Epoch 6/10
60000/60000 [==============================] - 4s 72us/sample - loss: 0.0665 - sparse_categorical_accuracy: 0.9789
Epoch 7/10
60000/60000 [==============================] - 4s 72us/sample - loss: 0.0568 - sparse_categorical_accuracy: 0.9812
Epoch 8/10
60000/60000 [==============================] - 4s 72us/sample - loss: 0.0516 - spars

In [ ]:
#เหตุผลที่ แบบ distubuted ช้ากว่าเนื่องจาก การเทรน แบบ distubuted นั้นเหมาะกับ GPU (หลายตัวด้วย)  